## An exploration of piq and HINT footprints on chr21

I reported that our piq footprints (for 18 samples and ~550 motifs) totaled > 2B, but HINT footprints were only about 20M: two orders of magnitude.   Seth helpfully explained that we should expect to see a pretty good match of HINT and piq results if we filtered the piq results on some one or more of the scores piq reports.  I proposed that I do some exploratory (visual) data analysis of this matter, asked Seth for some thresholds.   Seth's reply:

<code>
Two approaches seem reasonable for filtering the PIQ results to get a smaller number of high-confidence footprints.

1. The authors of PIQ recommend performing filtering based on the purity score. "Purity" is an odd term for 1 - FDR. i.e., a purity score of 0.7 corresponds to a 30% (predicted) False Discovery Rate. I think this is fine, but I'm not sure I trust their purity score will really function as an FDR...

2. Alternatively, we can use the comparison to ChIP-seq to guide us.  It is common to define a cutoff value for the predictor that achieves a balance between precision and recall (i.e., the "precision-recall break even" point). Note that we can apply this approach to scores from Wellington and HINT, as well as PIQ. I calculated the PRBE for each of the three footprinting algorithms. For PIQ, I used the "score" column in my analysis, since this combines evidence about both the motif sequence and the shape of the DNase. The PRBE point will differ between TFs, so we'll have to see if there is a reasonable value to use across all the TFs with ChIP-seq. Here are the PRBE values for ELF1:

         AUROC              PRBE        Precision.PRBE
fimo    0.6566       14.62802959     0.124315387000103
well.   0.8938       19.14902115     0.591500347532519
hint    0.9742       228.1427216     0.710662422916252
piq     0.7833       40.51518076     0.5462218250422
</code>

This notebook
   * makes connections to temporary databases (chr21 only) for HINT and piq
   * displays comparision igv tracks conditioned on region and scores
   

In [3]:
import pandas as pd, psycopg2 as psql

ImportError: dlopen(/Users/paul/anaconda/lib/python3.5/site-packages/psycopg2/_psycopg.cpython-35m-darwin.so, 2): Library not loaded: libssl.1.0.0.dylib
  Referenced from: /Users/paul/anaconda/lib/python3.5/site-packages/psycopg2/_psycopg.cpython-35m-darwin.so
  Reason: image not found